This code will load the model information, generate the model definition, and run the model estimation using FSL

In [ ]:
import nipype.algorithms.modelgen as model   # model generation
from  nipype.interfaces import fsl, ants      
from nipype.interfaces.base import Bunch
import os,json,glob,sys
import numpy
import nibabel
import nilearn.plotting

%matplotlib inline
import matplotlib.pyplot as plt

datadir='/home/jovyan/ClassData/'
    
results_dir = '/home/jovyan/ClassData/LabResults'#os.path.abspath("../../results")
if not os.path.exists(results_dir):
    os.mkdir(results_dir)

from nipype.caching import Memory
mem = Memory(base_dir='.')

print('Using data from',datadir)

In [ ]:
from bids import BIDSLayout
layout = BIDSLayout(datadir,validate=False)
layout

In [ ]:
#Psych60 Now lets look at what our data layout is a little closer.
#BIDS format is described in more depth here http://bids.neuroimaging.io/
#Please go and read up a little on BIDS and why it is important

In [ ]:
layout.get_subjects()

In [ ]:
layout.get_tasks()

In [ ]:
source_epi=layout.get(task="mental", extensions="nii")[0]
layout.get(task="mental", extensions="nii")[0].path

In [ ]:
#What does this tell us about this particular file?
source_epi.entities

In [ ]:
#Psych 60 please put answers inline
#What happens if you change the [0] above to another number?
layout.get(task="mental", extensions="nii")[0].path

In [ ]:
#Psych 60 
#What does the number signify?


In [ ]:
#Now please upload some sample onsets into your ClassData directory 
#then change onsets.txt to reflect the file you uploaded. 
#The files should be comma separated - this can be changed in Excel when you do Save As, CSV UFT-8 (Comma delimited)

In [ ]:
import pandas as pd
events1 = pd.read_csv(os.path.join(datadir, "onsets1.csv"), sep=",")
events2 = pd.read_csv(os.path.join(datadir, "onsets2.csv"), sep=",")
events3 = pd.read_csv(os.path.join(datadir, "onsets3.csv"), sep=",")

In [ ]:
offset=600 #10 minute runs
offset

In [ ]:
events2

In [ ]:
#make offsets for our runs
events2.onset=events2.onset+offset
events3.onset=events3.onset+offset*2

In [ ]:
frames=[events1,events2,events3]
events=pd.concat(frames)

In [ ]:
events

In [ ]:
#Do you have the correct number of events?

In [ ]:
for trial_type in events.trial_type.unique():
    print(events[events.trial_type == trial_type])

In [ ]:
#Psych60
#What did the above code do with the loop? Does this look correct?

In [ ]:
events

In [ ]:
#Psych60
#Please save notebook before proceeding!

In [ ]:
confoundfiles=layout.get(task="mental", extensions="tsv",subject='sid000006')
confoundfiles[0]

In [ ]:
confounds1=pd.read_csv(confoundfiles[0].path,sep="\t", na_values="n/a")
confounds1

In [ ]:
confounds1=pd.read_csv(confoundfiles[0].path,sep="\t", na_values="n/a")
confounds2=pd.read_csv(confoundfiles[1].path,sep="\t", na_values="n/a")
confounds3=pd.read_csv(confoundfiles[2].path,sep="\t", na_values="n/a")
frames=[confounds1,confounds2,confounds3]
confounds=pd.concat(frames)

In [ ]:
#Psych60 - please answer inline
#What regressors are we using? 
#What types of trials are we looking at?
#What type of noise do they account for? Google is your friend for this part...

info = [Bunch(conditions=['Social',
                          'Event',
                          'School',
                          'Career'],
              onsets=[list(events[events1.trial_type == 'Social'].onset),
                      list(events[events1.trial_type == 'Event'].onset),
                      list(events[events1.trial_type == 'School'].onset),
                      list(events[events1.trial_type == 'Career'].onset)],
              durations=[list(events[events1.trial_type == 'Social'].duration),
                          list(events[events1.trial_type == 'Event'].duration),
                          list(events[events1.trial_type == 'School'].duration),
                          list(events[events1.trial_type == 'Career'].duration)],
             regressors=[list(confounds1.FramewiseDisplacement.fillna(0)),
                         list(confounds1.aCompCor00),
                         list(confounds1.aCompCor01),
                         list(confounds1.aCompCor02),
                         list(confounds1.aCompCor03),
                         list(confounds1.aCompCor04),
                         list(confounds1.aCompCor05),
                        ],
             regressor_names=['FramewiseDisplacement',
                              'aCompCor00',
                              'aCompCor01',
                              'aCompCor02',
                              'aCompCor03',
                              'aCompCor04',
                              'aCompCor05',])
       ]

In [ ]:
#get all preprocessed files for subject 6
source_epi=layout.get(task="mental", extensions="nii", subject='sid000006')
source_epi

In [ ]:
%whos

In [ ]:
#This part defines what is brain and what is not and if we wanted we could trim timepoints we didn't want by changing t_min 
#skip = mem.cache(fsl.ExtractROI)
#skip_results1 = skip(in_file=source_epi[0].path,t_min=0, t_size=-1)
#skip_results2 = skip(in_file=source_epi[1].path,t_min=0, t_size=-1)
#skip_results3 = skip(in_file=source_epi[2].path,t_min=0, t_size=-1)

In [ ]:
#Please read down to the bottom of this and answer questions before running:
#What other ways are we removing noise?
#Any other special parameters that we are using in the model?

filelist=[source_epi[0].path, source_epi[1].path, source_epi[2].path]
s = model.SpecifyModel()
s.inputs.input_units = 'secs'
s.inputs.functional_runs = source_epi[0].path#filelist#skip_results.outputs.roi_file
s.inputs.time_repetition = 1 #1 second TR #layout.get_metadata(source_epi.filename)["RepetitionTime"]
s.inputs.high_pass_filter_cutoff = 128.
s.inputs.subject_info = info
specify_model_results = s.run()
s.inputs

In [ ]:
#This sets up contrasts for each condition individually and then compares Lips vs others
#Contrasts are the relative weight of the parameter estimates (betas) for each condition

social_cond = ['Social','T', ['Social'],[1]]
event_cond = ['Event','T', ['Event'],[1]]
school_cond = ['School','T', ['School'],[1]]
career_cond = ['School','T', ['School'],[1]]
school_vs_others = ["School vs. others",'T', ['School', 'Event', 'Social','Career'],[1, -1/3, -1/3, -1/3]]

In [ ]:
#Psych 60
#please write your own code for career_vs_others here
#by copying and pasting the code above to below this line and modifying it


In [ ]:
#Setting up one more contrast, an F contrast, think ANOVA

#all_task = ["All", 'F', [social_cond, event_cond, school_cond]]
contrasts=[social_cond, event_cond, school_cond, career_cond, school_vs_others]#, all_task]
#[social_cond, event_cond, school_cond, career_cond, school_vs_others, career_vs_others, all_task]
           
level1design = mem.cache(fsl.model.Level1Design)
level1design_results = level1design(interscan_interval = 1,#layout.get_metadata(source_epi.filename)["RepetitionTime"],
                                    bases = {'dgamma':{'derivs': True}},
                                    session_info = specify_model_results.outputs.session_info,
                                    model_serial_correlations=True,
                                    contrasts=contrasts)

level1design_results.outputs

In [ ]:
modelgen = mem.cache(fsl.model.FEATModel)
modelgen_results = modelgen(fsf_file=level1design_results.outputs.fsf_files,
                            ev_files=level1design_results.outputs.ev_files)
modelgen_results.outputs

In [ ]:
# This shows our study design
# What do each of the columns represent?
desmtx=numpy.loadtxt(modelgen_results.outputs.design_file,skiprows=5)
plt.imshow(desmtx,aspect='auto',interpolation='nearest',cmap='gray')

In [ ]:
#Correlation matrix of our regressors 
#are any of them highly correlated? Would this be a problem?

cc=numpy.corrcoef(desmtx.T)
plt.imshow(cc,aspect='auto',interpolation='nearest', cmap=plt.cm.viridis)
plt.colorbar()

In [ ]:
#mask = mem.cache(fsl.maths.ApplyMask)
#mask_results = mask(in_file=skip_results.outputs.roi_file,
#                    mask_file=os.path.join(datadir, "derivatives", "fmriprep", 
#                                        "sub-%s"%source_epi.subject, "ses-%s"%source_epi.session, "func", 
#                                        "sub-%s_ses-%s_task-fingerfootlips_bold_space-mni152nlin2009casym_brainmask.nii.gz"%(source_epi.subject,
#                                                                                                                             source_epi.session)))
#mask_results.outputs

In [ ]:
skip = mem.cache(fsl.ExtractROI)
skip_results1 = skip(in_file=source_epi[0].path,t_min=500, t_size=-1)

In [ ]:
#mask=layout.get(space='MNI152NLin2009cAsym',suffix='brainmask',extension="nii.gz")

In [ ]:
mask = mem.cache(fsl.maths.ApplyMask)
mask_results = mask(in_file=source_epi[0].path,mask_file='/home/jovyan/ClassData/sub-sid000006/func/sub-sid000006_task-mental_run-01_bold_space-MNI152NLin2009cAsym_brainmask.nii.gz')

In [ ]:
mask_results.outputs.outfile #show us the filename of our masked EPI

In [ ]:
#This section takes a LONG time. This may be as far as we make it during class today.
#While this is running - how many timepoints does it say we have? How many timeseries?
#Once this is done, how long did it take in total? How long do we need to allow to run all 3 runs for all participants?

#Actually fit the GLM to our data: see https://fsl.fmrib.ox.ac.uk/fsl/fslwiki/FEAT/UserGuide#FILM_General_Linear_Model

filmgls = mem.cache(fsl.FILMGLS)
filmgls_results = filmgls(in_file=mask_results.outputs.out_file,
                          design_file = modelgen_results.outputs.design_file,
                          tcon_file = modelgen_results.outputs.con_file,
                          fcon_file = modelgen_results.outputs.fcon_file,
                          autocorr_noestimate = True)
filmgls_results.outputs

In [ ]:
#This smooths each t map from our first run and displays them
for t_map in filmgls_results.outputs.zstats:
    nilearn.plotting.plot_glass_brain(nilearn.image.smooth_img(t_map, 8),
                                      display_mode='lyrz', colorbar=True, plot_abs=False, threshold=2.3)

In [ ]:
#fstat
#for t_map in [filmgls_results.outputs.zfstats]:
#    nilearn.plotting.plot_glass_brain(nilearn.image.smooth_img(t_map, 8),
#                                      display_mode='lyrz', colorbar=True, plot_abs=False, threshold=2.3)

In [ ]:
filmgls_results.outputs.copes

In [ ]:
#What are we talking about with copes in the code section below? 
#How did you define this in the script above?

In [ ]:
for t_map in filmgls_results.outputs.copes:
    nilearn.plotting.plot_glass_brain(nilearn.image.smooth_img(t_map, 8),
                                      display_mode='lyrz', colorbar=True, plot_abs=False, vmax=30)

In [ ]:
# How are the t-stats different from the cope? See slides 37-60
#https://fsl.fmrib.ox.ac.uk/fslcourse/lectures/feat1_part2.pdf
for t_map in filmgls_results.outputs.tstats:
    nilearn.plotting.plot_stat_map(nilearn.image.smooth_img(t_map, 8), colorbar=True, threshold=2.3)